# 🧠 Vector Store e Embeddings: A Memória Inteligente da IA

**Módulo 9 - Curso LangChain v0.3**

*Por Pedro Guth*

![](https://s3.us-east-1.amazonaws.com/turing.education/notebooks/imagens/langchain-modulo-09_img_01.png)

Bora entender como a IA consegue "lembrar" e encontrar informações relevantes de forma inteligente! 🚀

Até agora no nosso curso já vimos:
- Como conversar com LLMs (ChatModel)
- Como criar prompts inteligentes (Prompt Templates) 
- Como carregar e dividir documentos (Document Loading e Splitters)

Agora vamos aprender como fazer a IA "lembrar" dessas informações de forma eficiente!

## 🤔 Tá, mas o que é um Vector Store?

Imagina que você tem uma biblioteca gigante com milhões de livros. Como você encontraria rapidamente todos os livros sobre "culinária italiana"?

Um **Vector Store** é como um bibliotecário super inteligente que:
1. Lê cada livro e entende o "significado" dele
2. Organiza os livros não por ordem alfabética, mas por **similaridade de conteúdo**
3. Quando você pergunta algo, ele encontra os livros mais relevantes instantaneamente

### Como funciona na prática:
- Cada texto é convertido em **números** (vetores)
- Textos similares ficam "próximos" no espaço matemático
- Busca por similaridade ao invés de palavras exatas

**Dica!** Vector Stores são essenciais para RAG (Retrieval Augmented Generation) que vamos ver no próximo módulo!

## 🎯 E o que são Embeddings?

**Embeddings** são a mágica por trás dos Vector Stores!

Pensa assim: como você explicaria para um computador a diferença entre "cachorro" e "gato"? 

Embeddings fazem exatamente isso - eles convertem **palavras e textos em números** que capturam o significado semântico.

### Exemplo conceitual:
- "Rei" - "Homem" + "Mulher" = "Rainha"
- "Paris" - "França" + "Brasil" = "Brasília"

### Representação matemática:
$$\text{Embedding} = [0.1, -0.3, 0.7, 0.2, \ldots, 0.5]$$

Cada texto vira um vetor com centenas ou milhares de dimensões!

**Dica!** Similaridade é calculada usando distância coseno: $$\text{similarity} = \frac{A \cdot B}{|A| \times |B|}$$

In [ ]:
# Vamos começar instalando as bibliotecas que vamos usar
!pip install langchain langchain-community langchain-openai faiss-cpu sentence-transformers python-dotenv

# Imports básicos
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliotecas instaladas com sucesso!")

## 🔧 Setup do Ambiente LangChain

Agora vamos configurar nosso ambiente para trabalhar com embeddings no LangChain. Vamos usar modelos gratuitos para começar!

In [ ]:
# Imports do LangChain para embeddings e vector stores
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Vamos usar um modelo de embedding gratuito e bom em português
print("🔄 Carregando modelo de embeddings...")
embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

print("✅ Modelo carregado! Este modelo entende português e mais 49 idiomas!")
print(f"Dimensões do vetor: {len(embeddings_model.embed_query('teste'))}")

## 🧪 Primeiro Experimento: Visualizando Embeddings

Vamos ver como palavras similares ficam próximas no espaço vetorial!

In [ ]:
# Vamos testar com algumas palavras relacionadas
palavras = [
    "cachorro", "gato", "animal", "pet",
    "carro", "bicicleta", "transporte", "veículo",
    "pizza", "macarrão", "comida", "restaurante",
    "programação", "código", "python", "desenvolvimento"
]

print("🔄 Gerando embeddings para as palavras...")
embeddings_list = []
for palavra in palavras:
    embedding = embeddings_model.embed_query(palavra)
    embeddings_list.append(embedding)
    
# Convertendo para numpy array para facilitar os cálculos
embeddings_array = np.array(embeddings_list)
print(f"✅ Gerados {len(embeddings_list)} embeddings de {len(embeddings_list[0])} dimensões cada")

In [ ]:
# Vamos reduzir as dimensões para conseguir visualizar em 2D
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(embeddings_array)

# Criando o gráfico
plt.figure(figsize=(12, 8))
colors = ['red', 'red', 'red', 'red',      # animais
         'blue', 'blue', 'blue', 'blue',   # transporte
         'green', 'green', 'green', 'green', # comida
         'orange', 'orange', 'orange', 'orange'] # programação

for i, (palavra, cor) in enumerate(zip(palavras, colors)):
    plt.scatter(embeddings_2d[i, 0], embeddings_2d[i, 1], c=cor, s=100)
    plt.annotate(palavra, (embeddings_2d[i, 0], embeddings_2d[i, 1]), 
                xytext=(5, 5), textcoords='offset points', fontsize=10)

plt.title('Visualização de Embeddings - Palavras Similares Ficam Próximas!', fontsize=14)
plt.xlabel('Dimensão 1')
plt.ylabel('Dimensão 2')
plt.grid(True, alpha=0.3)
plt.legend(['Animais', 'Transporte', 'Comida', 'Programação'])
plt.show()

print("🎯 Liiindo! Veja como palavras do mesmo contexto ficam agrupadas!")

## 📊 Calculando Similaridade

Agora vamos calcular a similaridade entre palavras usando a **distância coseno**. Quanto mais próximo de 1, mais similares são as palavras!

In [ ]:
# Calculando matriz de similaridade
similarity_matrix = cosine_similarity(embeddings_array)

# Vamos ver alguns exemplos interessantes
print("🔍 SIMILARIDADES MAIS INTERESSANTES:")
print("="*50)

for i in range(len(palavras)):
    for j in range(i+1, len(palavras)):
        sim = similarity_matrix[i][j]
        if sim > 0.5:  # Só mostra similaridades altas
            print(f"{palavras[i]:12} ↔ {palavras[j]:12} | Similaridade: {sim:.3f}")

print("\n💡 Valores próximos de 1 = muito similares")
print("💡 Valores próximos de 0 = pouco similares")

## 🗃️ Criando nosso Primeiro Vector Store

Agora vamos criar um Vector Store de verdade! Vamos simular uma base de conhecimento sobre tecnologia.

Lembra do módulo anterior onde aprendemos sobre Document Loading? Agora vamos usar esses conceitos!

In [ ]:
# Vamos criar alguns documentos sobre tecnologia
documentos_texto = [
    "Python é uma linguagem de programação interpretada, de alto nível e de propósito geral.",
    "Machine Learning é um ramo da inteligência artificial que usa algoritmos para aprender padrões.",
    "O LangChain é um framework para desenvolvimento de aplicações com Large Language Models.",
    "JavaScript é a linguagem de programação mais popular para desenvolvimento web frontend.",
    "Deep Learning usa redes neurais artificiais com múltiplas camadas para resolver problemas complexos.",
    "React é uma biblioteca JavaScript para construção de interfaces de usuário interativas.",
    "Natural Language Processing permite que computadores entendam e processem linguagem humana.",
    "AWS é a plataforma de computação em nuvem mais popular do mundo.",
    "Docker é uma plataforma que usa containers para facilitar o deployment de aplicações.",
    "Git é um sistema de controle de versão distribuído usado para gerenciar código fonte."
]

# Convertendo textos em objetos Document (como vimos no módulo anterior)
documents = []
for i, texto in enumerate(documentos_texto):
    doc = Document(
        page_content=texto,
        metadata={"id": i, "categoria": "tecnologia"}
    )
    documents.append(doc)

print(f"✅ Criados {len(documents)} documentos para indexar no vector store")

In [ ]:
# Agora vamos criar nosso Vector Store usando FAISS
print("🔄 Criando Vector Store...")
print("Isso pode demorar um pouquinho na primeira vez...")

# FAISS é uma biblioteca super eficiente para busca por similaridade
vector_store = FAISS.from_documents(
    documents=documents,
    embedding=embeddings_model
)

print("✅ Vector Store criado com sucesso!")
print(f"📚 Indexados {len(documents)} documentos")
print("🎯 Agora podemos fazer buscas semânticas!")

## 🔍 Fazendo Buscas Semânticas

Agora vem a parte mais legal! Vamos fazer perguntas e ver como o Vector Store encontra documentos relevantes baseado no **significado**, não apenas nas palavras exatas.

In [ ]:
# Vamos fazer algumas consultas interessantes
consultas = [
    "Como funciona inteligência artificial?",
    "Qual linguagem usar para sites?", 
    "Como fazer deploy de aplicações?",
    "Ferramentas para gerenciar código",
    "Desenvolvimento de interfaces web"
]

for consulta in consultas:
    print(f"\n🔍 BUSCA: '{consulta}'")
    print("="*60)
    
    # Busca pelos 2 documentos mais similares
    docs_similares = vector_store.similarity_search(consulta, k=2)
    
    for i, doc in enumerate(docs_similares, 1):
        print(f"{i}. {doc.page_content}")
        
    print("\n💡 Veja como encontrou documentos relevantes mesmo sem usar as palavras exatas!")

## 📈 Busca com Scores de Similaridade

Vamos ver exatamente **quão similares** são os documentos encontrados através dos scores de similaridade.

In [ ]:
# Fazendo busca com scores de similaridade
consulta_teste = "programação e desenvolvimento de software"

print(f"🎯 ANÁLISE DETALHADA DA BUSCA: '{consulta_teste}'")
print("="*70)

# similarity_search_with_score retorna documentos E seus scores
docs_com_score = vector_store.similarity_search_with_score(consulta_teste, k=5)

for i, (doc, score) in enumerate(docs_com_score, 1):
    print(f"\n{i}. SCORE: {score:.4f}")
    print(f"   TEXTO: {doc.page_content}")
    
    # Vamos interpretar o score
    if score < 0.5:
        interpretacao = "🟢 Muito relevante"
    elif score < 1.0:
        interpretacao = "🟡 Relevante"
    else:
        interpretacao = "🔴 Pouco relevante"
    
    print(f"   RELEVÂNCIA: {interpretacao}")

print("\n📊 No FAISS, scores MENORES = mais similares (é uma distância!)")

## ⚡ Diferentes Tipos de Vector Stores

Existem vários tipos de Vector Stores, cada um com suas vantagens:

### 📋 Principais opções no LangChain:
- **FAISS**: Rápido, local, ótimo para prototipagem
- **Chroma**: Persistente, fácil de usar
- **Pinecone**: Gerenciado, escalável (pago)
- **Weaviate**: Open source, recursos avançados
- **Qdrant**: Russo, muito rápido

**Dica!** Para produção, considere options que persistem dados automaticamente!

In [ ]:
# Vamos também testar com Chroma (se disponível)
try:
    from langchain_community.vectorstores import Chroma
    
    print("🔄 Testando Chroma Vector Store...")
    
    # Criando vector store com Chroma
    chroma_store = Chroma.from_documents(
        documents=documents[:3],  # Só os primeiros 3 para testar
        embedding=embeddings_model
    )
    
    # Testando uma busca
    resultado = chroma_store.similarity_search("linguagem de programação", k=2)
    
    print("✅ Chroma funcionando!")
    print("🔍 Resultado da busca no Chroma:")
    for doc in resultado:
        print(f"   • {doc.page_content}")
        
except Exception as e:
    print(f"⚠️ Chroma não disponível: {e}")
    print("💡 Isso é normal! FAISS já é suficiente para aprender")

## 💾 Persistindo e Carregando Vector Stores

Na vida real, você não quer recriar seu Vector Store toda vez! Vamos aprender a salvar e carregar.

In [ ]:
# Salvando nosso Vector Store
import os

# Criando diretório para salvar
save_directory = "meu_vector_store"
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

print("💾 Salvando Vector Store...")

# FAISS permite salvar localmente
vector_store.save_local(save_directory)

print(f"✅ Vector Store salvo em: {save_directory}")
print(f"📁 Arquivos criados: {os.listdir(save_directory)}")

In [ ]:
# Carregando Vector Store salvo
print("📂 Carregando Vector Store salvo...")

# Carregando de volta
vector_store_carregado = FAISS.load_local(
    save_directory,
    embeddings_model,
    allow_dangerous_deserialization=True  # Necessário para versões mais recentes
)

print("✅ Vector Store carregado com sucesso!")

# Testando se funciona
teste = vector_store_carregado.similarity_search("inteligência artificial", k=1)
print(f"🧪 Teste: {teste[0].page_content}")

print("\n🎯 Liiindo! Agora você pode salvar e carregar seus Vector Stores!")

## 🔄 Adicionando Novos Documentos

Vector Stores não são estáticos! Você pode adicionar novos documentos dinamicamente.

In [ ]:
# Vamos adicionar alguns documentos novos
novos_textos = [
    "Kubernetes é uma plataforma de orquestração de containers open-source.",
    "FastAPI é um framework web moderno e rápido para construir APIs com Python.",
    "Pandas é uma biblioteca Python essencial para análise e manipulação de dados."
]

# Convertendo em Documents
novos_docs = []
for i, texto in enumerate(novos_textos):
    doc = Document(
        page_content=texto,
        metadata={"id": len(documents) + i, "categoria": "tecnologia", "novo": True}
    )
    novos_docs.append(doc)

print(f"➕ Adicionando {len(novos_docs)} novos documentos...")

# Adicionando ao vector store existente
vector_store.add_documents(novos_docs)

print("✅ Documentos adicionados com sucesso!")

# Testando busca com os novos documentos
resultado = vector_store.similarity_search("análise de dados", k=2)
print("\n🔍 Testando busca com novos documentos:")
for doc in resultado:
    print(f"   • {doc.page_content}")

## 🎨 Visualizando o Vector Store Completo

Vamos criar uma visualização legal de todo nosso vector store para entender como os documentos se organizam no espaço vetorial.

In [ ]:
# Vamos buscar todos os embeddings do nosso vector store
# Para isso, vamos fazer uma busca bem genérica que retorne muitos documentos
todos_docs = vector_store.similarity_search("tecnologia programação", k=20)

print(f"📊 Coletando embeddings de {len(todos_docs)} documentos...")

# Gerando embeddings de todos os documentos para visualizar
doc_embeddings = []
doc_labels = []

for doc in todos_docs:
    # Pegando só as primeiras palavras para o label
    label = ' '.join(doc.page_content.split()[:3]) + '...'
    doc_labels.append(label)
    
    # Gerando embedding
    embedding = embeddings_model.embed_query(doc.page_content)
    doc_embeddings.append(embedding)

# Reduzindo dimensionalidade para visualizar
doc_embeddings_array = np.array(doc_embeddings)
pca_docs = PCA(n_components=2)
docs_2d = pca_docs.fit_transform(doc_embeddings_array)

print("✅ Embeddings coletados! Criando visualização...")

In [ ]:
# Criando visualização dos documentos no espaço vetorial
plt.figure(figsize=(15, 10))

# Definindo cores baseadas no tipo de tecnologia
cores_por_tema = []
for label in doc_labels:
    if any(word in label.lower() for word in ['python', 'javascript', 'linguagem']):
        cores_por_tema.append('red')
    elif any(word in label.lower() for word in ['machine', 'deep', 'artificial', 'natural']):
        cores_por_tema.append('blue')
    elif any(word in label.lower() for word in ['react', 'web', 'interface']):
        cores_por_tema.append('green')
    elif any(word in label.lower() for word in ['aws', 'docker', 'kubernetes']):
        cores_por_tema.append('orange')
    else:
        cores_por_tema.append('gray')

# Plotando os pontos
for i, (x, y) in enumerate(docs_2d):
    plt.scatter(x, y, c=cores_por_tema[i], s=100, alpha=0.7)
    plt.annotate(doc_labels[i], (x, y), xytext=(5, 5), 
                textcoords='offset points', fontsize=8, 
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))

plt.title('Mapa do Vector Store - Documentos Similares Ficam Próximos!', fontsize=16)
plt.xlabel('Dimensão 1 (PCA)', fontsize=12)
plt.ylabel('Dimensão 2 (PCA)', fontsize=12)
plt.grid(True, alpha=0.3)

# Criando legenda
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='red', label='Linguagens de Programação'),
    Patch(facecolor='blue', label='Inteligência Artificial'),
    Patch(facecolor='green', label='Desenvolvimento Web'),
    Patch(facecolor='orange', label='Infraestrutura/Cloud'),
    Patch(facecolor='gray', label='Outros')
]
plt.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.show()

print("🎯 Liiindo! Veja como documentos similares se agrupam naturalmente!")

## 🛠️ Exercício Prático 1: Criando seu Vector Store

Agora é sua vez! Vamos criar um vector store sobre um tema diferente.

In [ ]:
# 🎯 EXERCÍCIO: Complete o código abaixo
# Crie um vector store sobre culinária brasileira

# 1. Crie uma lista com pelo menos 5 frases sobre culinária brasileira
culinaria_textos = [
    # Adicione suas frases aqui!
    "Feijoada é um dos pratos mais tradicionais da culinária brasileira.",
    # ... adicione mais 4 frases
]

# 2. Converta em Documents
# Seu código aqui...

# 3. Crie o vector store
# Seu código aqui...

# 4. Teste com uma busca sobre "sobremesa brasileira"
# Seu código aqui...

print("🍽️ Complete este exercício para praticar!")

## 🔗 Integrando com Chains (Preparação para RAG)

Lembra das Chains que vimos no módulo 6? Agora vamos conectar vector stores com chains, preparando o terreno para RAG no próximo módulo!

In [ ]:
# Criando uma chain simples que usa vector store
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import PromptTemplate

# Função simples para buscar documentos relevantes
def buscar_documentos(query):
    docs = vector_store.similarity_search(query, k=2)
    # Juntando o conteúdo dos documentos
    contexto = "\n".join([doc.page_content for doc in docs])
    return contexto

# Criando um template que usa o contexto do vector store
template = """Com base no seguinte contexto sobre tecnologia:

CONTEXTO:
{contexto}

PERGUNTA: {pergunta}

Responda de forma clara e baseada no contexto fornecido.
RESPOSTA:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["contexto", "pergunta"]
)

print("🔗 Chain preparada para usar com vector store!")
print("💡 No próximo módulo vamos ver como fazer isso com LLMs de verdade!")

# Testando a busca
pergunta_teste = "O que é Python?"
contexto_encontrado = buscar_documentos(pergunta_teste)

print(f"\n🔍 Para a pergunta: '{pergunta_teste}'")
print(f"📚 Contexto encontrado:\n{contexto_encontrado}")

## ⚙️ Configurações Avançadas de Vector Stores

Vamos ver algumas configurações mais avançadas que você pode usar para otimizar seus vector stores.

In [ ]:
# Testando diferentes algoritmos de busca
print("🔬 COMPARANDO DIFERENTES ALGORITMOS DE BUSCA")
print("="*60)

query_teste = "desenvolvimento de aplicações web"

# 1. Busca padrão por similaridade
docs_similarity = vector_store.similarity_search(query_teste, k=3)
print("\n1️⃣ SIMILARITY SEARCH (padrão):")
for i, doc in enumerate(docs_similarity, 1):
    texto_resumido = doc.page_content[:50] + "..."
    print(f"   {i}. {texto_resumido}")

# 2. Busca com MMR (Maximum Marginal Relevance)
# MMR balanceia relevância e diversidade
try:
    docs_mmr = vector_store.max_marginal_relevance_search(query_teste, k=3)
    print("\n2️⃣ MMR SEARCH (mais diverso):")
    for i, doc in enumerate(docs_mmr, 1):
        texto_resumido = doc.page_content[:50] + "..."
        print(f"   {i}. {texto_resumido}")
except:
    print("\n2️⃣ MMR não disponível nesta versão")

print("\n💡 MMR é útil quando você quer resultados relevantes MAS diversos!")

## 📊 Métricas e Avaliação de Vector Stores

Como saber se seu vector store está funcionando bem? Vamos criar algumas métricas simples!

In [ ]:
# Criando função para avaliar qualidade das buscas
def avaliar_vector_store(vector_store, queries_teste, respostas_esperadas):
    """
    Avalia a qualidade de um vector store
    """
    scores = []
    
    print("📊 AVALIANDO QUALIDADE DO VECTOR STORE")
    print("="*50)
    
    for i, (query, palavras_esperadas) in enumerate(zip(queries_teste, respostas_esperadas)):
        # Busca o documento mais relevante
        docs = vector_store.similarity_search(query, k=1)
        
        if docs:
            documento_encontrado = docs[0].page_content.lower()
            
            # Conta quantas palavras-chave esperadas aparecem no resultado
            palavras_encontradas = 0
            for palavra in palavras_esperadas:
                if palavra.lower() in documento_encontrado:
                    palavras_encontradas += 1
            
            score = palavras_encontradas / len(palavras_esperadas)
            scores.append(score)
            
            print(f"\n🔍 Query {i+1}: '{query}'")
            print(f"📄 Documento: {docs[0].page_content[:60]}...")
            print(f"🎯 Score: {score:.2f} ({palavras_encontradas}/{len(palavras_esperadas)} palavras-chave)")
        else:
            scores.append(0)
    
    score_medio = sum(scores) / len(scores) if scores else 0
    print(f"\n📊 SCORE MÉDIO DO VECTOR STORE: {score_medio:.2f}")
    
    return score_medio, scores

# Definindo queries de teste
queries_teste = [
    "linguagem para programar",
    "inteligência artificial e aprendizado", 
    "desenvolvimento web frontend"
]

# Palavras que esperamos encontrar em cada resposta
respostas_esperadas = [
    ["python", "javascript", "linguagem"],
    ["machine", "learning", "artificial", "deep"],
    ["javascript", "react", "web", "frontend"]
]

# Executando avaliação
score_final, scores_individuais = avaliar_vector_store(vector_store, queries_teste, respostas_esperadas)

## 🎯 Exercício Final: Vector Store Completo

Hora do exercício final! Vamos criar um vector store completo do zero.

In [ ]:
# 🎯 EXERCÍCIO FINAL: Vector Store sobre Esportes
# Sua missão: criar um sistema completo de busca sobre esportes

print("🏆 EXERCÍCIO FINAL: Sistema de Busca sobre Esportes")
print("="*60)
print("\n📝 TAREFAS:")
print("1. Criar pelo menos 8 documentos sobre diferentes esportes")
print("2. Criar o vector store")
print("3. Fazer 3 buscas diferentes")
print("4. Salvar o vector store")
print("5. Adicionar 2 novos documentos")
print("6. Testar uma busca final")

# Comece aqui!
# Dica: use esportes como futebol, basquete, tênis, natação, etc.

# 1. Seus documentos sobre esportes:
esportes_docs = [
    # Complete com pelo menos 8 frases sobre esportes!
]

print("\n🚀 Complete este exercício para dominar vector stores!")
print("💡 Dica: pense em diferentes aspectos - regras, equipamentos, história...")

## 🎊 Resumo e Próximos Passos

**Parabéns! Você dominou Vector Stores e Embeddings! 🎉**

### 📚 O que aprendemos:
- ✅ **Embeddings**: Como transformar texto em vetores numéricos
- ✅ **Vector Stores**: Como armazenar e buscar por similaridade semântica
- ✅ **FAISS**: Vector store rápido para prototipagem
- ✅ **Busca Semântica**: Encontrar documentos por significado, não palavras exatas
- ✅ **Persistência**: Salvar e carregar vector stores
- ✅ **Métricas**: Avaliar a qualidade das buscas

### 🔄 Conexão com o curso:
- **Módulos anteriores**: Usamos Documents do módulo 8
- **Próximo módulo**: RAG (Retrieval Augmented Generation) - onde vamos conectar vector stores com LLMs!

### 🎯 **No próximo módulo (RAG)** vamos:
- Conectar vector stores com Gemini/ChatGPT
- Fazer perguntas e obter respostas baseadas em nossos documentos
- Criar um sistema completo de Q&A

**Dica!** Vector Stores são a base de quase todas as aplicações modernas de IA que trabalham com documentos!

![](https://s3.us-east-1.amazonaws.com/turing.education/notebooks/imagens/langchain-modulo-09_img_02.png)